In [74]:
import cv2
recognizer = cv2.face.LBPHFaceRecognizer_create()
import numpy as np
import os
import scipy.io as sio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import ggmm.gpu as ggmm

In [75]:
import sys, glob, argparse
import numpy as np
import math
from scipy.stats import multivariate_normal
import time
from sklearn import svm

num_of_images = 10000
# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def image_vgg_desc(file):
    data = sio.loadmat(file)
    return data['features']

def image_vgg_desc_averaged(file, n):
    data = sio.loadmat(file)
    data = data['features']
    avg_data = [sum(data[0][i:i+n])//n for i in range(0,len(data[0]),n)]
    avg_data = np.array(avg_data).reshape((1, len(avg_data)))
    #print('data shape - ', avg_data.shape)
    return avg_data

def folder_descriptors(folder):
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    print("Calculating descriptos. Number of images is", len(files))
    return np.concatenate([image_vgg_desc_averaged(file, 4) for file in files])


def generate_gmm(input_folder):
    words = np.concatenate([folder_descriptors(input_folder)]) 
    print("Training GMM of size, input size - ", words.shape)
    
    N, D = words.shape
    K = 32 #128 #256 #4096 #128

    ggmm.init()
    gmm = ggmm.GMM(K,D)

    thresh = 1e-3 # convergence threshold
    n_iter = 20 # maximum number of EM iterations
    init_params = 'wmc' # initialize weights, means, and covariances

    # train GMM
    gmm.fit(words, thresh, n_iter, init_params=init_params)
    
    weights = gmm.get_weights()
    means = gmm.get_means()
    covs = gmm.get_covars()

    print("dict done")

    np.save("means_100.gmm", means)
    np.save("covs_100.gmm", covs)
    np.save("weights_100.gmm", weights)
    print("GMM is done")
    return means, covs, weights

GPU is active


In [76]:


m, c, w = generate_gmm('gen_data/vgg_resize/')

Calculating descriptos. Number of images is 10000
Training GMM of size, input size -  (10000, 1024)
dict done
GMM is done


In [77]:


def likelihood_moment(x, ytk, moment):
    x_moment = np.power(np.float32(x), moment) if moment > 0 else np.float32([1])
    return x_moment * ytk

def likelihood_statistics(samples, means, covs, weights):
    gaussians, s0, s1,s2 = {}, {}, {}, {}
    #print(samples.shape)
    samples = zip(range(0, len(samples)), samples)

    g = [multivariate_normal(mean=means[k], cov=covs[k]) for k in range(0, len(weights)) ]
    for index, x in samples:
        gaussians[index] = np.array([g_k.pdf(x) for g_k in g])

    for k in range(0, len(weights)):
        s0[k], s1[k], s2[k] = 0, 0, 0
        for index, x in samples:
            probabilities = np.multiply(gaussians[index], weights)
            probabilities = probabilities / np.sum(probabilities)
            s0[k] = s0[k] + likelihood_moment(x, probabilities[k], 0)
            s1[k] = s1[k] + likelihood_moment(x, probabilities[k], 1)
            s2[k] = s2[k] + likelihood_moment(x, probabilities[k], 2)

    return s0, s1, s2

def fisher_vector_weights(s0, s1, s2, means, covs, w, T):
    return np.float32([((s0[k] - T * w[k]) / np.sqrt(w[k]) ) for k in range(0, len(w))])

def fisher_vector_means(s0, s1, s2, means, sigma, w, T):
    #print(len(s0), len(s1), len(s2), len(w))
    #print(means[0].shape, sigma[0].shape)
    a = [(s1[k] - means[k][:256] * s0[k]) for k in range(0, len(w))]
    b = [(np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))]
    #return np.float32(a/b)
    return np.float32([(s1[k] - means[k][:256] * s0[k]) / (np.sqrt(w[k] * sigma[0][k])) for k in range(0, len(w))])

def fisher_vector_sigma(s0, s1, s2, means, sigma, w, T):
    return np.float32([(s2[k] - 2 * means[k][:256]*s1[k]  + (means[k][:256]*means[k][:256] - sigma[0][k]) * s0[k]) 
                       / (np.sqrt(2*w[k])*sigma[0][k])  for k in range(0, len(w))])

def normalize(fisher_vector):
    v = np.sqrt(abs(fisher_vector)) * np.sign(fisher_vector)
    #print('in normalize')
    return v / np.sqrt(np.dot(v, v))

def fisher_vector(samples, means, covs, w):
    #return 0
    s0, s1, s2 =  likelihood_statistics(samples, means, covs, w)
    T = samples.shape[0]
    #print('in fisher _vector likelihood done')
    #covs = np.float32([np.diagonal(covs[k]) for k in range(0, covs.shape[0])])
    covs = np.float32([np.diagonal(covs)])
    a = fisher_vector_weights(s0, s1, s2, means, covs, w, T)
    b = fisher_vector_means(s0, s1, s2, means, covs, w, T)
    c = fisher_vector_sigma(s0, s1, s2, means, covs, w, T)
    #print('outputs -', a.shape, b[0].shape, c.shape)
    #fv = np.concatenate([np.concatenate(a), np.concatenate(b[0]), np.concatenate(c[0])])
    fv = np.concatenate([a, b[0], c[0]])
    fv = normalize(fv)
    print('image - ', time.time())
    #img_count = img_count + 1
    return fv

"""
def get_fisher_vectors_from_folder(folder, means, covs, w):
    files = glob.glob(folder + "/*.mat")
    files = files[:num_of_images]
    #print (files)
    return np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w) for file in files])
"""
def fisher_features(folder, means, covs, w):
    outpath = 'gen_data/fv_1024/'
    print('means shape - ', means.shape)
    print('covs shape - ', covs.shape)
    print('weights shape - ', w.shape)
    files = sorted(glob.glob(folder + "/*.mat"))
    files = files[:num_of_images]
    
    count = 1
    for file in files:
        fv = np.float32([fisher_vector(image_vgg_desc_averaged(file, 4), means, covs, w)])
        f = file.split('/')
        f = f[2].split('.')
        print("saving the file ", f[0] + '.fv', "count - ", count)
        count = count + 1
        sio.savemat(outpath + f[0] +'.fv', {'fv':fv}, do_compression=True)
    #features = {'vgg_resize' : get_fisher_vectors_from_folder(folder, means, covs, w)}

    return

In [78]:
t0 = time.time()
fisher_features('gen_data/vgg_resize/', m, c, w)
print('fisher_features are generated')
print("Extracting training fisher vectors for 10000 images took %0.3fs." % (time.time() - t0))

means shape -  (32, 1024)
covs shape -  (32, 1024)
weights shape -  (32,)
image -  1544935592.3594599
saving the file  10.fv count -  1
image -  1544935594.4979692
saving the file  100009.fv count -  2
image -  1544935596.653287
saving the file  100021.fv count -  3
image -  1544935598.7796898
saving the file  100032.fv count -  4
image -  1544935600.8934908
saving the file  100034.fv count -  5
image -  1544935602.9902275
saving the file  100036.fv count -  6
image -  1544935605.181097
saving the file  100039.fv count -  7
image -  1544935607.3120868
saving the file  10004.fv count -  8
image -  1544935609.3956997
saving the file  100043.fv count -  9
image -  1544935611.4673505
saving the file  100048.fv count -  10
image -  1544935613.546277
saving the file  100061.fv count -  11
image -  1544935615.5995605
saving the file  100073.fv count -  12
image -  1544935617.6488738
saving the file  100087.fv count -  13
image -  1544935619.753039
saving the file  100088.fv count -  14
image 

image -  1544935850.2928772
saving the file  100701.fv count -  123
image -  1544935852.3983605
saving the file  100718.fv count -  124
image -  1544935854.4275613
saving the file  100721.fv count -  125
image -  1544935856.465805
saving the file  100745.fv count -  126
image -  1544935858.5186605
saving the file  100750.fv count -  127
image -  1544935860.6220372
saving the file  100758.fv count -  128
image -  1544935862.6562111
saving the file  100760.fv count -  129
image -  1544935864.6617815
saving the file  100767.fv count -  130
image -  1544935866.6804626
saving the file  100771.fv count -  131
image -  1544935868.7070432
saving the file  100773.fv count -  132
image -  1544935870.7419674
saving the file  100776.fv count -  133
image -  1544935872.7922122
saving the file  100779.fv count -  134
image -  1544935874.8243763
saving the file  100784.fv count -  135
image -  1544935876.7956254
saving the file  100790.fv count -  136
image -  1544935878.7857964
saving the file  1008

image -  1544936102.6072226
saving the file  101385.fv count -  245
image -  1544936104.7477496
saving the file  101392.fv count -  246
image -  1544936106.8450282
saving the file  101411.fv count -  247
image -  1544936108.9669118
saving the file  101420.fv count -  248
image -  1544936111.0618007
saving the file  101423.fv count -  249
image -  1544936113.2479181
saving the file  101429.fv count -  250
image -  1544936115.3641896
saving the file  101432.fv count -  251
image -  1544936117.490488
saving the file  101435.fv count -  252
image -  1544936119.6069057
saving the file  101436.fv count -  253
image -  1544936121.7097504
saving the file  101440.fv count -  254
image -  1544936123.8172367
saving the file  101444.fv count -  255
image -  1544936125.9280374
saving the file  101447.fv count -  256
image -  1544936128.0502002
saving the file  101451.fv count -  257
image -  1544936130.138981
saving the file  101455.fv count -  258
image -  1544936132.238331
saving the file  101458

image -  1544936358.6215844
saving the file  102130.fv count -  367
image -  1544936360.714611
saving the file  102131.fv count -  368
image -  1544936362.8377075
saving the file  102134.fv count -  369
image -  1544936364.9715507
saving the file  102138.fv count -  370
image -  1544936367.1196349
saving the file  102140.fv count -  371
image -  1544936369.209083
saving the file  102149.fv count -  372
image -  1544936371.2782776
saving the file  102151.fv count -  373
image -  1544936373.3536096
saving the file  102154.fv count -  374
image -  1544936375.4394588
saving the file  10216.fv count -  375
image -  1544936377.4156625
saving the file  102170.fv count -  376
image -  1544936379.4548903
saving the file  102171.fv count -  377
image -  1544936381.565398
saving the file  102176.fv count -  378
image -  1544936383.5817804
saving the file  10218.fv count -  379
image -  1544936385.6296043
saving the file  102180.fv count -  380
image -  1544936387.658139
saving the file  102187.fv

image -  1544936614.9115226
saving the file  102845.fv count -  489
image -  1544936616.9176376
saving the file  102849.fv count -  490
image -  1544936618.981827
saving the file  102853.fv count -  491
image -  1544936621.0975223
saving the file  102858.fv count -  492
image -  1544936623.220067
saving the file  102863.fv count -  493
image -  1544936625.3661437
saving the file  102870.fv count -  494
image -  1544936627.4789262
saving the file  102871.fv count -  495
image -  1544936629.5384834
saving the file  102883.fv count -  496
image -  1544936631.5835304
saving the file  102886.fv count -  497
image -  1544936633.7584388
saving the file  102888.fv count -  498
image -  1544936635.9807508
saving the file  10289.fv count -  499
image -  1544936637.991294
saving the file  102896.fv count -  500
image -  1544936640.0637271
saving the file  102904.fv count -  501
image -  1544936642.1720433
saving the file  102905.fv count -  502
image -  1544936644.249355
saving the file  102913.f

...

image -  1544956736.864467
saving the file  154119.fv count -  9507
image -  1544956739.2007873
saving the file  154125.fv count -  9508
image -  1544956741.5445948
saving the file  154127.fv count -  9509
image -  1544956743.9156446
saving the file  15413.fv count -  9510
image -  1544956746.2647636
saving the file  154149.fv count -  9511
image -  1544956748.6476912
saving the file  154152.fv count -  9512
image -  1544956751.017423
saving the file  154160.fv count -  9513
image -  1544956753.4399433
saving the file  154167.fv count -  9514
image -  1544956755.81219
saving the file  154171.fv count -  9515
image -  1544956758.1478488
saving the file  154184.fv count -  9516
image -  1544956760.5052912
saving the file  154189.fv count -  9517
image -  1544956762.8508098
saving the file  154193.fv count -  9518
image -  1544956765.2203858
saving the file  154195.fv count -  9519
image -  1544956767.6182003
saving the file  154201.fv count -  9520
image -  1544956769.971543
saving the f

image -  1544957023.127264
saving the file  154771.fv count -  9627
image -  1544957025.4818966
saving the file  154776.fv count -  9628
image -  1544957027.8775513
saving the file  154786.fv count -  9629
image -  1544957030.2440164
saving the file  154788.fv count -  9630
image -  1544957032.5866911
saving the file  154792.fv count -  9631
image -  1544957034.9539719
saving the file  154818.fv count -  9632
image -  1544957037.324436
saving the file  154829.fv count -  9633
image -  1544957039.6944706
saving the file  154842.fv count -  9634
image -  1544957042.0432184
saving the file  154846.fv count -  9635
image -  1544957044.3923368
saving the file  154856.fv count -  9636
image -  1544957046.7704623
saving the file  154864.fv count -  9637
image -  1544957049.1442082
saving the file  154877.fv count -  9638
image -  1544957051.505174
saving the file  154882.fv count -  9639
image -  1544957053.8727381
saving the file  154904.fv count -  9640
image -  1544957056.2440696
saving th

image -  1544957308.4420552
saving the file  155485.fv count -  9747
image -  1544957310.8445764
saving the file  155506.fv count -  9748
image -  1544957313.2420468
saving the file  155508.fv count -  9749
image -  1544957315.6177104
saving the file  155509.fv count -  9750
image -  1544957317.9974074
saving the file  155517.fv count -  9751
image -  1544957320.3528564
saving the file  15552.fv count -  9752
image -  1544957322.7165601
saving the file  155531.fv count -  9753
image -  1544957325.1169932
saving the file  155536.fv count -  9754
image -  1544957327.478456
saving the file  155537.fv count -  9755
image -  1544957329.8602624
saving the file  155542.fv count -  9756
image -  1544957332.2151902
saving the file  155557.fv count -  9757
image -  1544957334.5880811
saving the file  155558.fv count -  9758
image -  1544957336.953263
saving the file  15558.fv count -  9759
image -  1544957339.3109257
saving the file  155602.fv count -  9760
image -  1544957341.6649833
saving the

image -  1544957594.7224727
saving the file  156291.fv count -  9867
image -  1544957597.127674
saving the file  156292.fv count -  9868
image -  1544957599.5269156
saving the file  156296.fv count -  9869
image -  1544957601.9118016
saving the file  1563.fv count -  9870
image -  1544957604.3128483
saving the file  156305.fv count -  9871
image -  1544957606.7242398
saving the file  156312.fv count -  9872
image -  1544957609.1321945
saving the file  156316.fv count -  9873
image -  1544957611.4904468
saving the file  15632.fv count -  9874
image -  1544957613.8620994
saving the file  156320.fv count -  9875
image -  1544957616.2344162
saving the file  156326.fv count -  9876
image -  1544957618.5870588
saving the file  156330.fv count -  9877
image -  1544957620.9534278
saving the file  156334.fv count -  9878
image -  1544957623.308937
saving the file  156348.fv count -  9879
image -  1544957625.6810591
saving the file  156352.fv count -  9880
image -  1544957628.070115
saving the f

image -  1544957881.6998386
saving the file  156957.fv count -  9987
image -  1544957884.155538
saving the file  156970.fv count -  9988
image -  1544957886.5511544
saving the file  156972.fv count -  9989
image -  1544957888.948652
saving the file  156997.fv count -  9990
image -  1544957891.3381388
saving the file  157000.fv count -  9991
image -  1544957893.7459192
saving the file  157002.fv count -  9992
image -  1544957896.147277
saving the file  157003.fv count -  9993
image -  1544957898.547837
saving the file  15702.fv count -  9994
image -  1544957900.9255714
saving the file  157021.fv count -  9995
image -  1544957903.272753
saving the file  157023.fv count -  9996
image -  1544957905.6326866
saving the file  157031.fv count -  9997
image -  1544957908.024595
saving the file  157048.fv count -  9998
image -  1544957910.4060059
saving the file  157055.fv count -  9999
image -  1544957912.797249
saving the file  157057.fv count -  10000
fisher_features are generated
Extracting 

In [ ]:
#CSV file reading
"""
Generating dictionaries
for photo_id ---> buisness_id
"""
photo_to_bus_dict = {}
with open('train_photo_to_biz_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            photo_to_bus_dict[row[0]] = row[1]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')
print(len(photo_to_bus_dict))
    
"""
Generating dictionaries
for business_id ---> labels
"""
bus_to_labels_dict = {}     
with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            bus_to_labels_dict[row[0]] = [row[1]]
#             print(f'\t{row[0]} works in the {row[1]} department')
            line_count += 1
    print(f'Processed {line_count} lines.')  
print(len(bus_to_labels_dict))